In [ ]:
function GibbsSamplerERU(y,X,xpx,b,niter,νB,Sb)
    chain = zeros(niter)   #to save sigma2
    n,k = size(X)
    vare = 1.0
    varb = Sb*νB/(νB - 2)
    ycorr = y - X*b      # adjusting y for all covariates 
    @showprogress "drawing MCMC samples..." for iter=1:niter    
        # sampling intercept
        x = X[:,1]
        rhs = x'ycorr + xpx[1]*b[1]
        #rhs = dot(x,ycorr) + xpx[1]*b[1]
        lhsi = 1/xpx[1]
        bHat = lhsi*rhs
        oldb = b[1]
        b[1] = rand(Normal(bHat, sqrt(lhsi))) # using residual var = 1 
        #BLAS.axpy!(oldb-b[1],x,ycorr)
        ycorr += (oldb - b[1])*x
        λ = vare/varb
        # sampling marker effects
        for i=2:k
            x = X[:,i]
            rhs = x'ycorr + xpx[i]*b[i]
            lhsi = 1/(xpx[i] + λ)
            bHat = lhsi*rhs
            oldb = b[i]
            b[i] = rand(Normal(bHat, sqrt(lhsi)))
            ycorr += (oldb - b[i])*x
            #BLAS.axpy!(oldb-b[i],x,ycorr)
        end
        # sampling sigmaB
        varb = (sum(b[2:end].^2) + νB*Sb) /rand(Chisq(νB+k-1))
        chain[iter] = varb
    end
    return chain
end